## Implementation of Likelihood Weighting to approximate probability queries from a Bayesian Network.

In [2]:
import numpy as np
from calculations_helper import break_up_polytree, handle_dag_likelihood_weighting, join_distributions

def estimate_likelihood_weighting(iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Use likelihood weighting to estimate the probability distribution of the given query variables

    Args:
        iterations (int): number of iterations where the variables are sampled and their counts are updated accordingly
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables and their values

    Returns:
        np.array: resulting probability distribution estimate for the query variables
    """
    dag_map, query_collections, evidence_collections = break_up_polytree(network, queries, evidence)
    
    if len(dag_map) == 1:
        # only one directed acyclic graph
        return queries, handle_dag_likelihood_weighting(iterations, network, queries, evidence)
    else:
        # each directed acyclic graph will output a probability distribution - we must join them all and keep track of the variables present
        reordered_queries = []
        prob_distributions = []
        for i, dag in dag_map.items():
            these_queries = query_collections[i]
            for v in these_queries:
                reordered_queries.append(v)
            this_evidence = {v:evidence[v] for v in evidence_collections[i]}
            prob_distributions.append(handle_dag_likelihood_weighting(iterations, dag, these_queries, this_evidence))
        return reordered_queries, join_distributions(prob_distributions)

In [3]:
import json

queries = [0]
evidence = {3:True,4:True}

with open('bn_test_1.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

([0], array([0.71907143, 0.28092857]))


In [4]:
import json

queries = [0, 3]
evidence = {2:True}

with open('bn_test_2.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

([0, 3], array([0.27130039, 0.04670004, 0.13639991, 0.54559965]))


In [5]:
import json

queries = [1]
evidence = {2:False}

with open('bn_test_3.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

([1], array([0.87794749, 0.12205251]))


In [6]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('small_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

([1, 3], array([7.22792169e-04, 9.36394791e-01, 2.95154573e-03, 5.99308709e-02]))


In [7]:
import json

queries = [1, 3]
evidence = {2:False, 5:True}

with open('big_polytree.json') as f:
    bayesian_network = json.load(f)
    print(estimate_likelihood_weighting(10000, bayesian_network, queries, evidence))

([1, 3], array([0.73475573, 0.07968897, 0.16854277, 0.01701253]))
